# 機械学習フレームワーク チュートリアル

このノートブックでは、機械学習フレームワークの基本的な使い方を紹介します。データの前処理から様々なモデルでの学習、評価、交差検証、ハイパーパラメータチューニングまでの一連の流れを説明します。

## 1. 初期設定

必要なモジュールをインポートします。

In [ ]:
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing

# フレームワークのコンポーネントをインポート
from src.data.loader import DataLoader
from src.data.base_preprocessor import StandardPreprocessor
from src.models.supervised.regression.linear import LinearRegression
from src.training.base_trainer import SupervisedTrainer
from src.training.cross_validation import CrossValidator
from src.training.hyperparameter_tuning import create_tuner
from src.utils.logger import setup_logger

# ロガーの設定
logger = setup_logger("tutorial", "INFO")

## 2. サンプルデータの準備

ボストン住宅価格データセットを使用します。

In [ ]:
# ボストン住宅価格データセットを読み込み
boston = load_boston()
data = pd.DataFrame(boston.data, columns=boston.feature_names)
data["target"] = boston.target

# データの先頭を確認
data.head()

## 3. 設定の準備

フレームワークで使用する設定を準備します。

In [ ]:
# 実験用の設定
config = {
    "data": {"raw_dir": "../data/raw", "processed_dir": "../data/processed", "target_col": "target"},
    "preprocessing": {"numeric_cols": list(boston.feature_names), "categorical_cols": [], "drop_cols": []},
    "training": {"random_state": 42, "test_size": 0.2},
    "model": {"type": "linear_regression", "fit_intercept": True},
}

## 4. データの分割と前処理

データをトレーニングセットとテストセットに分割し、前処理を行います。

In [ ]:
# DataLoaderの初期化
data_loader = DataLoader(config)

# データの分割
X_train, X_test, y_train, y_test = data_loader.split_data(
    data,
    target_col="target",
    test_size=config["training"]["test_size"],
    random_state=config["training"]["random_state"],
)

print(f"トレーニングデータのサイズ: {X_train.shape}")
print(f"テストデータのサイズ: {X_test.shape}")

In [ ]:
# 前処理
preprocessor = StandardPreprocessor(config["preprocessing"])
X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_test_processed = preprocessor.transform(X_test)

# 前処理後のデータを確認
X_train_processed.head()

## 5. モデルのトレーニングと評価

線形回帰モデルを使用して、データをトレーニングします。

In [ ]:
# モデルの初期化
model = LinearRegression(config["model"])

# トレーナーの初期化
trainer = SupervisedTrainer(model, config["training"])

# モデルのトレーニング
trainer.train(X_train_processed, y_train)

# モデルの評価
metrics = trainer.evaluate(X_test_processed, y_test)
print("\nモデル評価結果:")
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

## 6. 交差検証の実行

モデルの安定性をチェックするために交差検証を行います。

In [ ]:
# 交差検証の設定
cv_config = {"cv_type": "kfold", "scoring": "r2", "cv_params": {"n_splits": 5, "shuffle": True, "random_state": 42}}

# 新しいモデルを作成（学習状態をリセット）
cv_model = LinearRegression(config["model"])

# 交差検証の実行
cross_validator = CrossValidator(cv_model, cv_config)
cv_results = cross_validator.cross_validate(X_train_processed, y_train)

print("\n交差検証結果:")
for name, value in cv_results.items():
    if isinstance(value, float):
        print(f"{name}: {value:.4f}")

## 7. 学習曲線の生成

学習曲線を生成して、モデルの学習パターンを確認します。

In [ ]:
# 学習曲線の生成
learning_curve_results = cross_validator.learning_curve(X_train_processed, y_train, plot=True)

## 8. ハイパーパラメータチューニング

グリッドサーチを使用してモデルのハイパーパラメータをチューニングします。

In [ ]:
# チューニングの設定
tuning_config = {
    "tuner_type": "grid",
    "cv": 5,
    "scoring": "r2",
    "param_grid": {"fit_intercept": [True, False], "positive": [False, True]},
}

# 新しいモデルを作成
tune_model = LinearRegression(config["model"])

# チューナーの作成と実行
tuner = create_tuner("grid", tune_model, tuning_config)
tuning_results = tuner.tune(X_train_processed, y_train)

print("\nハイパーパラメータチューニング結果:")
print(f"最適スコア: {tuning_results['best_score']:.4f}")
print("最適パラメータ:")
for name, value in tuning_results["best_params"].items():
    print(f"  {name}: {value}")

## 9. 最適化モデルの評価

チューニングで得られた最適なパラメータを使用してモデルを評価します。

In [ ]:
# チューニング済みモデルの評価
tuned_trainer = SupervisedTrainer(tuner.model, config["training"])
tuned_metrics = tuned_trainer.evaluate(X_test_processed, y_test)

print("\n最適化モデルの評価結果:")
for name, value in tuned_metrics.items():
    print(f"{name}: {value:.4f}")

## 10. モデルの保存と読み込み

トレーニングしたモデルを保存し、後で読み込めることを確認します。

In [ ]:
# モデルの保存
os.makedirs("../models/saved", exist_ok=True)
save_path = "../models/saved/linear_regression_tuned.pkl"
tuned_trainer.save_model(save_path)
print(f"モデルを保存しました: {save_path}")

# モデルの読み込み
loaded_model = LinearRegression.load(save_path)
loaded_trainer = SupervisedTrainer(loaded_model, config["training"])
loaded_metrics = loaded_trainer.evaluate(X_test_processed, y_test)

print("\n読み込んだモデルの評価結果:")
for name, value in loaded_metrics.items():
    print(f"{name}: {value:.4f}")

## まとめ

このチュートリアルでは、機械学習フレームワークの基本的な使い方を紹介しました。フレームワークを使用すると、データの前処理、モデルのトレーニング、評価、交差検証、ハイパーパラメータチューニングなどを一貫したインターフェースで行うことができます。